In [1]:
import facebook
import pandas as pd
import pymongo
import time
from pymongo import MongoClient
from transformers import pipeline
from datetime import datetime

connection = MongoClient("mongodb://localhost:27017")
db = connection.IndeedJobPostDB
df = pd.DataFrame(list(db.Job_Posts.find()))
df = df.sort_values(by=['Current_Date'], ascending=False)
df = df.reset_index(drop=True)
df

,_id,Unnamed: 0,JobID,Title,Job_Type,Company,Salary,Schedule,Location,Date_Posted,Current_Date,Details,Apply_link
0,64e461ba602f350b823dc0e0,0,NY14072023FRI001,FOOD SERVER ASSISTANT (BUSSER),Full-time,Park Hyatt New York,$17.29 - $23.06 an hour,NaN,"New York, NY 10019 (Midtown area)",Just posted,23-07-14,The Food Server Assistant is responsible prima...,https://www.indeed.com//rc/clk?jk=923ac23a84cb...
1,64e461ba602f350b823dc13e,94,TX14072023FRI1005,Host/Hostess,Part-time,Gyu-Kaku Japanese BBQ,NaN,Weekends as needed+2,"Houston, TX 77070 (Willowbrook area)",Just posted,23-07-14,Helpful attitude and friendly demeanor. He or ...,https://www.indeed.com//rc/clk?jk=9a6e26a88780...
2,64e461ba602f350b823dc140,96,TX14072023FRI1007,Host,Part-time,Denny's - Westheimer Road,NaN,Weekends as needed+1,"Houston, TX 77077",Just posted,23-07-14,"A Denny’s Host or Hostess has a “Guests First""...",https://www.indeed.com//rc/clk?jk=bcc00ac3cafb...
3,64e461ba602f350b823dc141,97,TX14072023FRI1008,Server,"Full-time, Part-time",Denny's - 6131 Highway 6 S,NaN,Weekends as needed+1,"Missouri City, TX 77459",Just posted,23-07-14,Prior experience in one of the following roles...,https://www.indeed.com//rc/clk?jk=81be6634db19...
4,64e461ba602f350b823dc142,98,TX14072023FRI1009,Server,"Full-time, Part-time",Denny's - W. Pasadena Fwy,NaN,Weekends as needed+1,"Pasadena, TX 77506",Just posted,23-07-14,Prior experience in one of the following roles...,https://www.indeed.com//rc/clk?jk=5114425a2fc5...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,64e461ba602f350b823dc112,50,NY14072023FRI3006,"Server - Brooklyn, NY",Full-time,Sushi Tokyo Inc,$20 - $45 an hour,NaN,"Brooklyn, NY 11223 (Gravesend-Sheepshead Bay a...",Today,23-07-14,Provide accurate orders to the kitchen staff. ...,https://www.indeed.com//rc/clk?jk=375a7aff9356...
146,64e461ba602f350b823dc113,51,NY14072023FRI3007,Food and Beverage Supervisor,Full-time,1 Hotel Brooklyn Bridge,$29 an hour,NaN,"Brooklyn, NY 11201",Today,23-07-14,That means hiring leaders for our Food & Bever...,https://www.indeed.com//rc/clk?jk=ee7acf1ff07e...
147,64e461ba602f350b823dc114,52,NY14072023FRI3008,Server Assistant (Earn up to $800/wk),Full-time,Au Cheval NYC,NaN,Weekends as needed+2,"New York, NY 11203 (East Flatbush area)",Today,23-07-14,Ofrecemos un conjunto completo de beneficios y...,https://www.indeed.com//rc/clk?jk=ffe1a20f22e0...
148,64e461ba602f350b823dc115,53,NY14072023FRI3009,Busperson - Conrad New York Downtown,NaN,Hilton,Estimated $39.4K - $49.8K a year,NaN,"New York, NY 10282 (Battery Park area)",Today,23-07-14,Respond to guest inquiries and requests and re...,https://www.indeed.com//rc/clk?jk=dd0d55f52262...


- Get Data
- Filter data
- set timer for posts
- Post at each group (based on location)

In [2]:
df.isna().sum()

_id              0
Unnamed: 0       0
JobID            0
Title            0
Job_Type        12
Company          0
Salary          79
Schedule        73
Location         0
Date_Posted      0
Current_Date     0
Details          0
Apply_link       0
dtype: int64

In [3]:
df = df[df.Schedule.notnull()]
df

,_id,Unnamed: 0,JobID,Title,Job_Type,Company,Salary,Schedule,Location,Date_Posted,Current_Date,Details,Apply_link
1,64e461ba602f350b823dc13e,94,TX14072023FRI1005,Host/Hostess,Part-time,Gyu-Kaku Japanese BBQ,NaN,Weekends as needed+2,"Houston, TX 77070 (Willowbrook area)",Just posted,23-07-14,Helpful attitude and friendly demeanor. He or ...,https://www.indeed.com//rc/clk?jk=9a6e26a88780...
2,64e461ba602f350b823dc140,96,TX14072023FRI1007,Host,Part-time,Denny's - Westheimer Road,NaN,Weekends as needed+1,"Houston, TX 77077",Just posted,23-07-14,"A Denny’s Host or Hostess has a “Guests First""...",https://www.indeed.com//rc/clk?jk=bcc00ac3cafb...
3,64e461ba602f350b823dc141,97,TX14072023FRI1008,Server,"Full-time, Part-time",Denny's - 6131 Highway 6 S,NaN,Weekends as needed+1,"Missouri City, TX 77459",Just posted,23-07-14,Prior experience in one of the following roles...,https://www.indeed.com//rc/clk?jk=81be6634db19...
4,64e461ba602f350b823dc142,98,TX14072023FRI1009,Server,"Full-time, Part-time",Denny's - W. Pasadena Fwy,NaN,Weekends as needed+1,"Pasadena, TX 77506",Just posted,23-07-14,Prior experience in one of the following roles...,https://www.indeed.com//rc/clk?jk=5114425a2fc5...
5,64e461ba602f350b823dc143,99,TX14072023FRI1010,Casual Dining Server,"Full-time, Part-time",TGI Friday's,NaN,Day shift+1,"Humble, TX 77338",Just posted,23-07-14,1 year server experience in a full-service res...,https://www.indeed.com//rc/clk?jk=31b7c78faaba...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,64e461ba602f350b823dc10d,45,NY14072023FRI3001,Busser,"Full-time, Part-time",Lucciola Italian Restaurant,NaN,Day shift+6,"New York, NY 10024 (Upper West Side area)",Today,23-07-14,"Assist in setting up and clearing tables, ensu...",https://www.indeed.com//company/Piccolo-Cafe/j...
141,64e461ba602f350b823dc10f,47,NY14072023FRI3003,Part Time Server for FINE DINING Restaurant,"Full-time, Part-time",Lucciola Italian Restaurant,NaN,Monday to Friday+8,"New York, NY 10024 (Upper West Side area)",Today,23-07-14,Ability to thrive in a fast-paced and high-pre...,https://www.indeed.com//company/Piccolo-Cafe/j...
143,64e461ba602f350b823dc110,48,NY14072023FRI3004,Host/Hostess,"Full-time, Part-time",DOMA Land + Sea,NaN,Monday to Friday+5,"Cedarhurst, NY 11516",Today,23-07-14,The host is expected to greet and seat our gue...,https://www.indeed.com//rc/clk?cmp=DOMA-Land-%...
147,64e461ba602f350b823dc114,52,NY14072023FRI3008,Server Assistant (Earn up to $800/wk),Full-time,Au Cheval NYC,NaN,Weekends as needed+2,"New York, NY 11203 (East Flatbush area)",Today,23-07-14,Ofrecemos un conjunto completo de beneficios y...,https://www.indeed.com//rc/clk?jk=ffe1a20f22e0...


In [4]:
df.isna().sum()

_id              0
Unnamed: 0       0
JobID            0
Title            0
Job_Type         4
Company          0
Salary          73
Schedule         0
Location         0
Date_Posted      0
Current_Date     0
Details          0
Apply_link       0
dtype: int64

In [4]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [6]:
summarizer(df.Summary[2], max_length = 150)[0]['summary_text']

" Emmy Squared Hell's Kitchen is hiring RESTAURANT SERVERS to join our team . Servers earn $15.00 per hour (this is a tipped position subject to a $5.00 tip credit if applicable). Full time employees are eligible for comprehensive health benefits, including medical, dental, vision, life insurance and more ."

In [20]:
len(df)

71

In [11]:
summary[0]

{'summary_text': 'our servers go the extra mile to make it memorable . you’ll help with menu selections, quickly refill beverages and ensure food arrives on time, always delivering warm and friendly interactions . our busy restaurants mean consistent hours, flexible schedules and the opportunity to make a great income – as well as a lot of friends and memories along the way . if you have less experience, consider cashier or barista / Baker .'}

In [12]:
def message(df, i, summary):
    return (
        f'**Test Post: {i}**\n'
        f'Post ID: {df.JobID[i]}\n'
        f'Job Opening: {df.Title[i]}\n'
        f'\n'
        f'{df.Location[i]}\n'
        f'\n'
        f'{df.Job_Type[i]}\n'
        f'\n'
        f'👉Comment "Interested" If You want to apply.\n'
        f'\n'
        f'Work Summary: \n'
        f"{summary}\n"
        f'\n'
        f'#houstonjobs #houston #ergool #houstonjob #jobsearch #jobseekers #search #empower #career #nowhiring #jobopportunities #jobplacement #employmentassistance #jobupdates #restaurantjobs #retailjobs #jobopening #jobalert'
    )

In [13]:
count = 0
print(len(df))
for row in range(len(df)):
    try:
        print(df['Details'][row])
    except:
        count += 1
        print(f"Key Error for row {row}")
        pass
print(count)

77
Key Error for row 0
Helpful attitude and friendly demeanor. He or she interacts with guests and staff in a cordial, efficient, and professional manner and helps with preparing to…
A Denny’s Host or Hostess has a “Guests First" attitude, engaging in friendly conversation with guests as you perform duties. Must be 16 or older.
Prior experience in one of the following roles/brands is highly desirable: server, servers, waiter, waiters, wait staff, runner, restaurant crew member,…
Prior experience in one of the following roles/brands is highly desirable: server, servers, waiter, waiters, wait staff, runner, restaurant crew member,…
1 year server experience in a full-service restaurant. No one has a bigger impact on the guest’s than you. You know the menu and bring fun to every table!
Prior experience in one of the following roles/brands is highly desirable: server, servers, waiter, waiters, wait staff, runner, restaurant crew member,…
A Denny’s Host or Hostess has a “Guests First" attitu

In [14]:
groups = ["674232410710265", "658314902622656"] #NY, TX
access_token = "EAAJHKZAZAZBPbsBO8niM1nsPQz23X70xBOofkcKhc7hMwHSLrE7UvTZBY29ZAfc608FBIYSpVZAZCe7tH6TZAkExuC1ZCcg2psobZBnjxQcqyZCk6ZAEK6mxbiUPKuUOIhSwidTu1b1x0rEyNTFeSaTpWHwa92pAT61oHDZBDgU4xsXd8AoyaykpnUlIRPCExul0sa9xjXne4PZB0f3AVNg6dsPFET1VAAqhTI6CUKZAklypC1OVgNR87g2uTxU39CZCxysZD"
graph = facebook.GraphAPI(access_token=access_token)

for row in range(len(df)):
    print(f'Post No. {row}')
    try:
        summary = df.Details[row]
        
        if "NY" in df.Location[row]:
            try:
                x = graph.put_object(groups[0], 'feed', message=message(df, row, summary))
                print(x)
                print("Posted Successfully in NY Group!")
                data_to_send = [datetime.now().date(), datetime.now().time(), df.JobID[row], df.Title[row], df.Company[row], df.Location[row], df.Apply_link[row]]  # Replace with your data
                send_data_to_google_sheet(data_to_send)
            except Exception as error:
                print("An exception occurred:", error)
                print(f"Passed the error and posted {df.JobID[row]}")
        elif "TX" in df.Location[row]:
            try:
                x = graph.put_object(groups[1], 'feed', message=message(df, row, summary))
                print(x)
                print("Posted Successfully in the TX Group!")
                data_to_send = [datetime.now().date(), datetime.now().time(), df.JobID[row], df.Title[row], df.Company[row], df.Location[row], df.Apply_link[row]]  # Replace with your data
                send_data_to_google_sheet(data_to_send)
            except Exception as error:
                print("An exception occurred:", error)
                print(f"Passed the error and posted {df.JobID[row]}")
    except Exception as error:
        print("An exception occurred:", error)
     
    
    time.sleep(10)

Post No. 0
An exception occurred: 0
Post No. 1
{'id': '658314902622656_832805545173590'}
Posted Successfully in the TX Group!
Post No. 2
{'id': '658314902622656_832805705173574'}
Posted Successfully in the TX Group!
Post No. 3
{'id': '658314902622656_832805798506898'}
Posted Successfully in the TX Group!
Post No. 4
{'id': '658314902622656_832805985173546'}
Posted Successfully in the TX Group!
Post No. 5
{'id': '658314902622656_832806135173531'}
Posted Successfully in the TX Group!
Post No. 6
{'id': '658314902622656_832806235173521'}
Posted Successfully in the TX Group!
Post No. 7
{'id': '658314902622656_832806365173508'}
Posted Successfully in the TX Group!
Post No. 8
{'id': '658314902622656_832806505173494'}
Posted Successfully in the TX Group!
Post No. 9
{'id': '658314902622656_832806595173485'}
Posted Successfully in the TX Group!
Post No. 10
An exception occurred: 10
Post No. 11
An exception occurred: 11
Post No. 12
An exception occurred: 12
Post No. 13
An exception occurred: 13
Po

- change the summary
- Job Title
- RPM(REquest per minute) 200 requests per hour
- https://developers.facebook.com/docs/graph-api/overview/rate-limiting/
- 1 app multiple users
- 75 posts each day
- Unique ID's for the Jobs(e.g. NYTIME)
- Write an excel file from Digital Ocean (Unique_ID, Status, Apply_Link)

In [15]:
import requests

base_url = 'https://graph.facebook.com/v13.0/oauth/access_token'

params = {
    'grant_type': 'fb_exchange_token',
    'client_id': '641193897508283',
    'client_secret': '05cd75c91b61a5ade15f62308ceadc2a',
    'fb_exchange_token': 'EAAJHKZAZAZBPbsBO76bpyx8F139ZA2utVrfrALg7M7H4kw55qfJgFIZBGcYzCAgIruTMDGOVnCSsncvUT6vhloM0YG5EEibkzKoQS45Q5Yhua5zkA7Gnmr96bN65fgSK8L2vVMG1QsU6KKgIfRBuZCxWNhLOEQiqZCiqo4Akv2VJ4Urk86bHXRhJlkBTOt5OYJApYdWuZCZAdBKcZCpk6Bx5BD0osUZAL0i6uaztZAoVANqNdvcA8lWSI6KJDA5meAZDZD'
}

response = requests.get(base_url, params=params)

if response.status_code == 200:
    response_data = response.json()
    print(response_data)
else:
    print(f"Failed to exchange token. Status code: {response.status_code}")


{'access_token': 'EAAJHKZAZAZBPbsBO5t4D0jmbuVZBM41QJlvw8OB7ZASADxZA5F1RsSPBmbzzvcZBPyVBrE7T5qhJeasJ2kkkWwJ6H9Dro96ZCbpuhBGT2ZB9urGe0M2wscFemWtZB6VyEot2ZBSihFX1jQ5ZAoK26m65eQ3TicbZBoENSsiwuxweTnRZBJKTehz9J3ZAqeAg0vZA', 'token_type': 'bearer', 'expires_in': 5174587}


In [1]:
import requests

url = 'https://graph.facebook.com/v13.0/me'
params = {'access_token': 'EAAJHKZAZAZBPbsBO5t4D0jmbuVZBM41QJlvw8OB7ZASADxZA5F1RsSPBmbzzvcZBPyVBrE7T5qhJeasJ2kkkWwJ6H9Dro96ZCbpuhBGT2ZB9urGe0M2wscFemWtZB6VyEot2ZBSihFX1jQ5ZAoK26m65eQ3TicbZBoENSsiwuxweTnRZBJKTehz9J3ZAqeAg0vZA'}

response = requests.get(url, params=params)

# Get the response status code
status_code = response.status_code

print(f"Response status code: {status_code}")

# Check if the request was successful (status code 200)
if status_code == 200:
    response_data = response.json()
    print(response_data)
else:
    print("Request failed.")


Response status code: 200
{'name': 'Rittique Alam', 'id': '6522012197888038'}


In [9]:
import os
import pickle
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
import json

# Read the JSON file
with open('FBGrpJobPosting.json', 'r') as json_file:
    creds = json.load(json_file)

# Define the scope and Google Sheets API version
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = 'YOUR_SPREADSHEET_ID'
RANGE_NAME = 'Daily_Group_Posts'  # Update with your sheet's name

with open('FBGrpJobPosting.json', 'r') as json_file:
        cred = json.load(json_file)
creds

{'web': {'client_id': '852044117694-vchj59ied0p1rgu77c4nrtdm22j5i9bi.apps.googleusercontent.com',
  'project_id': 'jobapp-eb06c',
  'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
  'token_uri': 'https://oauth2.googleapis.com/token',
  'auth_provider_x509_cert_url': 'https://www.googleapis.com/oauth2/v1/certs',
  'client_secret': 'YxP2V-Tisu8vy_Hwzqqhtf-E',
  'redirect_uris': ['https://jobapp-eb06c.firebaseapp.com/__/auth/handler'],
  'javascript_origins': ['http://localhost',
   'http://localhost:5000',
   'https://jobapp-eb06c.firebaseapp.com']}}

In [10]:
def get_credentials():
    creds = None
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

def send_data_to_google_sheet(data, creds):
    creds = creds
    service = build('sheets', 'v4', credentials=creds)

    values = [data]  # Assuming data is a list of values to be added

    body = {'values': values}
    result = service.spreadsheets().values().append(
        spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME,
        valueInputOption='RAW', body=body
    ).execute()

    print(f'{result.get("updates").get("updatedCells")} cells updated.')


data_to_send = ['Value1', 'Value2', 'Value3']  # Replace with your data
send_data_to_google_sheet(data_to_send, creds)

AttributeError: 'dict' object has no attribute 'authorize'

In [3]:
import os
from google.auth.transport.requests import Request
from google.oauth.credentials import Credentials
from google_auth_oauthlib.flow import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

SPREADSHEET_ID = "1nCT9zi1zic2SYKHsEMp9rm4rWA0bDTZcknXGWlZhrDc"
RANGE_NAME = 'Sheet1'

def getSheet():
    if os.path.exists("token.json"):
        credentials = Credentials.from_authorized_suer_file("token.json", SCOPES)
        
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            credentials = flow.run_local_server(port=0)
        with open("token.json") as token:
            token.write(credentials.to_json())
            
    try:
        service = build("sheets", "v4", credentials=credentials)
        sheets = service.spreadsheets()
        result = sheets.values().get(
        spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME
    ).execute()
        
        values = result.get('values', [])
    
        if not values:
            print('No data found.')
            return

        start_row = 1
        start_col = 1
        end_row = len(values)
        end_col = len(values[0])

        range_of_cells = f'{RANGE_NAME}!{chr(65+start_col)}{start_row}:{chr(65+end_col)}{end_row}'
        print(f"Non-empty cells range: {range_of_cells}")
    except HttpError as error:
        print(error)
        
getSheet()

ModuleNotFoundError: No module named 'google.oauth'